In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils import delete_nones, encode_ids

## Load and Preprocess data

In [ ]:
frame = pd.read_csv("../data/clear_data.scv")

In [ ]:
frame = delete_nones(frame)
frame = encode_ids(frame)

In [ ]:
# TODO: move into main
print('frame shape = ', frame.shape)
print('num unique drivers = ', np.array(frame.driver_id.unique()).shape[0])
print('num unique users = ', np.array(frame.user_id.unique()).shape[0])
print('num unique rides = ', np.array(frame.ride_id.unique()).shape[0])

## Compute Features

In [1]:
import xgboost as xgb
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import normalize
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

In [ ]:
# read in data
dtrain = xgb.DMatrix(np.array(X_train_add[[]]), 
                     label=np.array(y_train))

dtest = xgb.DMatrix(np.array(X_test_add[[]]),
                    label=np.array(y_test))

params_scope = [{'booster': 'dart',
                 'max_depth': 5, 
                 'learning_rate': 0.1,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.1,
                 'skip_drop': 0.5,
                 'num_class': 3},

                 {'booster': 'dart',
                 'max_depth': 3, 
                 'learning_rate': 0.01,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.1,
                 'skip_drop': 0.8,
                 'num_class': 3},

                 {'booster': 'gblinear',
                 'max_depth': 8, 
                 'learning_rate': 0.001,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'forest',
                 'num_class': 3},

                {'booster': 'dart',
                 'max_depth': 4, 
                 'learning_rate': 0.1,
                 'objective': 'multi:softmax', 
                 'silent': True,
                 'normalize_type': 'tree',
                 'rate_drop': 0.2,
                 'skip_drop': 0.6,
                 'num_class': 3}
                ]

In [ ]:
scores = []
scores_cv = []

for param_num, params in enumerate(params_scope):
    print()
    print('##############################')
    print('###           {}           ###'.format(param_num))
    print('##############################')
    bst = xgb.train(param, dtrain)
    
    y_pred = bst.predict(dtest)
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score)
    print('Score = ', score)
    
    
    score_cv = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    scores_cv.append(score_cv)
    
    print('score_cv = ', scores_cv)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import normalize
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

from utils import delete_nones, encode_ids
from features import time_discretization, find_hexagon_id_of_pickup_coords, create_city_hexagons

/home/mak7/PycharmProjects/MURA/mur/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# TODO: insert data cleaning
frame = pd.read_csv("./data/clear_data.scv")

In [3]:
# DATA PREPROCESSING
frame = delete_nones(frame)
frame = encode_ids(frame)
# delete canceled requests
frame = frame[(frame['canceled_by_client'] == 0) & (frame['canceled_by_driver'] == 0)]

frame shape =  (1117300, 12)
clear frame shape =  (995791, 12)


In [4]:
# get hexagon frame
# TODO: pass params of city
hexagon_frame = create_city_hexagons()

In [5]:
hexagon_frame.head()

,hexagon_id,latitude1,latitude2,latitude3,latitude4,latitude5,latitude6,longitude1,longitude2,longitude3,longitude4,longitude5,longitude6
0,1,49.843030,49.838817,49.835233,49.835861,49.840073,49.843658,24.025218,24.022414,24.027112,24.034614,24.037419,24.032721
1,2,49.838189,49.833976,49.830392,49.831020,49.835233,49.838817,24.014911,24.012108,24.016806,24.024308,24.027112,24.022414
2,3,49.835233,49.831020,49.827436,49.828064,49.832276,49.835861,24.027112,24.024308,24.029005,24.036506,24.039311,24.034614
3,4,49.845985,49.841772,49.838189,49.838817,49.843030,49.846614,24.013016,24.010212,24.014911,24.022414,24.025218,24.020519
4,5,49.847870,49.843658,49.840073,49.840700,49.844912,49.848497,24.035526,24.032721,24.037419,24.044921,24.047727,24.043029


In [6]:
# TODO: add city column
# FEATURE GENERATION
time_period_frame = time_discretization(frame=frame)
frame['week_day'], frame['weekend'], frame['month'], frame['year'] = time_period_frame['week_day'], \
                                                                 time_period_frame['weekend'], \
                                                                 time_period_frame['month'], \
                                                                 time_period_frame['year']

In [8]:
frame = frame.reset_index(drop=True)

In [9]:
frame.head()

,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0
3,3,2018-10-03 16:53:48.097000000,49.871124,23.957130,49.858883,24.019121,77669,1456,6.74,0,0,0,6.0,True,9.0,2018.0
4,4,2019-01-25 06:25:31.730000000,49.785679,24.055679,49.842602,24.000990,57120,3846,13.16,0,0,0,3.0,False,12.0,2018.0


In [31]:
from matplotlib import path
from tqdm import tqdm

In [47]:
def find_hexagon_id_of_pickup_coords(hexagon_frame, frame):
    # TODO: decide what to do with pickoff_lat and pickoff_lng
    pickup_coords = np.array(frame[['pickup_lat', 'pickup_lng']])

    hexagon_arr = np.array(hexagon_frame)[:, 1:]
    hexagon_ids = np.array(hexagon_frame['hexagon_id'])

    hexagon_pickup_ids = []

    for coord_ind, pickup_coord in tqdm(enumerate(pickup_coords)):
        # TODO: use kNN for optimization
        for hexagon_ind, hexagon in enumerate(hexagon_arr):
            hexagon = np.array([hexagon[0:6], hexagon[6:]]).reshape(-1, 6).T
            hexagon_path = path.Path(hexagon)

            if hexagon_path.contains_points([(pickup_coord[0], pickup_coord[1])]):
                hexagon_pickup_ids.append(hexagon_ids[hexagon_ind])
                break
            elif hexagon_ind == hexagon_arr.shape[0]-1:
                hexagon_pickup_ids.append(None)
            else:
                continue

    return hexagon_pickup_ids

In [71]:
small_frame = frame[0:15000]
print(small_frame.shape)
small_frame.head()

(15000, 16)


,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0
3,3,2018-10-03 16:53:48.097000000,49.871124,23.957130,49.858883,24.019121,77669,1456,6.74,0,0,0,6.0,True,9.0,2018.0
4,4,2019-01-25 06:25:31.730000000,49.785679,24.055679,49.842602,24.000990,57120,3846,13.16,0,0,0,3.0,False,12.0,2018.0


In [74]:
small_frame['hexagon_id'] = find_hexagon_id_of_pickup_coords(hexagon_frame, small_frame)


0it [00:00, ?it/s]
27it [00:00, 254.94it/s]
48it [00:00, 234.49it/s]
78it [00:00, 250.78it/s]
112it [00:00, 272.08it/s]
153it [00:00, 302.62it/s]
188it [00:00, 310.57it/s]
218it [00:00, 305.71it/s]
248it [00:00, 298.46it/s]
279it [00:00, 300.21it/s]
309it [00:01, 295.93it/s]
344it [00:01, 309.91it/s]
376it [00:01, 311.47it/s]
408it [00:01, 309.71it/s]
442it [00:01, 315.89it/s]
474it [00:01, 280.74it/s]
504it [00:01, 284.21it/s]
533it [00:01, 285.30it/s]
562it [00:01, 279.91it/s]
591it [00:02, 270.50it/s]
632it [00:02, 300.70it/s]
668it [00:02, 315.79it/s]
701it [00:02, 296.03it/s]
732it [00:02, 289.35it/s]
772it [00:02, 314.98it/s]
808it [00:02, 326.09it/s]
844it [00:02, 334.35it/s]
879it [00:02, 332.99it/s]
916it [00:02, 339.94it/s]
951it [00:03, 317.90it/s]
984it [00:03, 302.50it/s]
1017it [00:03, 309.98it/s]
1057it [00:03, 330.82it/s]
1101it [00:03, 353.29it/s]
1145it [00:03, 374.75it/s]
1184it [00:03, 350.08it/s]
1223it [00:03, 357.51it/s]
1260it [00:03, 336.57it/s]
1303it [00:04,

10557it [00:30, 361.82it/s]
10596it [00:30, 338.36it/s]
10639it [00:30, 361.12it/s]
10677it [00:30, 363.23it/s]
10723it [00:30, 387.42it/s]
10764it [00:30, 374.11it/s]
10803it [00:31, 361.25it/s]
10845it [00:31, 376.11it/s]
10887it [00:31, 386.28it/s]
10927it [00:31, 370.81it/s]
10965it [00:31, 356.11it/s]
11002it [00:31, 346.15it/s]
11038it [00:31, 324.65it/s]
11079it [00:31, 345.85it/s]
11124it [00:31, 371.50it/s]
11163it [00:32, 334.18it/s]
11203it [00:32, 349.11it/s]
11255it [00:32, 385.51it/s]
11296it [00:32, 389.27it/s]
11337it [00:32, 384.35it/s]
11387it [00:32, 412.48it/s]
11430it [00:32, 389.25it/s]
11471it [00:32, 365.58it/s]
11512it [00:32, 376.12it/s]
11568it [00:33, 415.94it/s]
11612it [00:33, 413.99it/s]
11655it [00:33, 391.26it/s]
11696it [00:33, 384.12it/s]
11736it [00:33, 387.11it/s]
11776it [00:33, 353.39it/s]
11820it [00:33, 374.53it/s]
11859it [00:33, 377.02it/s]
11898it [00:33, 373.48it/s]
11936it [00:34, 321.69it/s]
11973it [00:34, 333.23it/s]
12008it [00:34, 328.

In [75]:
small_frame.head()

,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year,hexagon_id
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0,23.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0,33.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0,1.0
3,3,2018-10-03 16:53:48.097000000,49.871124,23.957130,49.858883,24.019121,77669,1456,6.74,0,0,0,6.0,True,9.0,2018.0,NaN
4,4,2019-01-25 06:25:31.730000000,49.785679,24.055679,49.842602,24.000990,57120,3846,13.16,0,0,0,3.0,False,12.0,2018.0,NaN


In [76]:
small_frame = small_frame.dropna()
small_frame = small_frame

In [77]:
small_frame.head()

,ride_id,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,ride_to_suburb,week_day,weekend,month,year,hexagon_id
0,0,2018-12-14 23:00:00.043000000,49.824671,24.013246,49.784729,24.060303,45133,2726,7.97,0,0,0,4.0,False,12.0,2018.0,23.0
1,1,2018-08-25 21:40:52.537000000,49.839718,23.994425,49.840610,24.029135,122255,1942,3.17,0,0,0,4.0,False,1.0,2019.0,33.0
2,2,2019-01-18 09:18:44.377000000,49.837013,24.034357,49.832710,23.999216,20280,665,3.39,0,0,0,4.0,False,1.0,2019.0,1.0
5,5,2018-09-30 04:15:57.223000000,49.798859,24.061859,49.813759,23.959866,95136,3114,9.30,0,0,0,4.0,False,1.0,2019.0,109.0
6,6,2018-12-20 02:28:31.017000000,49.844044,24.024796,49.843155,23.972368,12655,4259,4.55,0,0,0,1.0,False,2.0,2019.0,6.0


In [120]:
features = small_frame.groupby(['week_day', 'month', 'year', 'hexagon_id']).sum()

In [121]:
features.head(100)

ride_id   pickup_lat   pickup_lng  \
week_day month year   hexagon_id                                      
0.0      1.0   2019.0 1.0          356996  2192.969907  1057.319612   
                      2.0           14304    49.833111    24.021141   
                      3.0           46089   348.824989   168.225653   
                      4.0           70719   448.569771   216.172186   
                      5.0           39939   249.211762   120.184799   
                      6.0          181528  1046.776146   504.570682   
                      7.0           29500   199.342205    96.160755   
                      8.0           17122    99.679008    48.099739   
                      9.0           25814   199.405410    96.211523   
                      10.0          26149    99.667048    48.098737   
                      11.0          38751   249.162243   120.045799   
                      12.0          44744   299.072971   144.034161   
                      13.0          19558   199.315948    96.088425   
                      14.0          23117   149.548992    72.063106   
                      15.0          26178   149.555795    72.110802   
                      16.0          40634   249.269587   120.135607   
                      17.0           9114    49.830048    24.046576   
                      18.0          45106   249.123993   120.186937   
                      19.0          27253   149.508343    72.027250   
                      20.0          11032    49.827095    23.999249   
                      21.0          57261   349.017429   168.115158   
                      22.0           8740    49.824265    24.047560   
                      23.0          86995   498.225891   240.110012   
                      26.0          53131   249.166168   119.994686   
                      27.0           8115    99.650833    48.108522   
                      31.0          26326    99.696129    47.988031   
                      32.0           6192    99.642788    48.047619   
                      33.0          82040   398.720358   191.960537   
                      34.0          32215   398.899242   192.181094   
                      37.0          21412    99.656723    48.137794   
...                                   ...          ...          ...   
                      116.0         28916   249.034992   119.901447   
                      122.0         62364   348.632179   167.950115   
                      124.0          7649    99.619087    47.938774   
                      126.0          9821    49.818386    23.973110   
                      127.0         14386    99.645943    47.925161   
         2.0   2019.0 1.0          142994  1046.638260   504.662025   
                      2.0           68441   548.176169   264.192695   
                      3.0           29581   249.155830   120.158993   
                      4.0           38390   199.371784    96.078752   
                      5.0           25961    99.685360    48.073223   
                      6.0          116388   747.679340   360.387899   
                      7.0           14332    99.670959    48.090136   
                      11.0          59915   249.155830   120.046770   
                      12.0          39659   249.227463   120.042326   
                      14.0          20637    99.698624    48.036596   
                      15.0          38108   199.414310    96.158747   
                      16.0          28225   249.269661   120.121172   
                      17.0          31457   249.141621   120.218830   
                      19.0          47684   199.351994    96.024048   
                      20.0          13992   199.301495    96.004244   
                      21.0          30739   199.433567    96.058454   
                      23.0          14576    99.641457    48.016115   
                      24.0           2188    49.835316    24.068558   
                      26.0          45628   249.161290   1

In [108]:
feature_frame.columns

Index(['week_day', 'month', 'year', 'hexagon_id', 'ride_id', 'created_at',
       'pickup_lat', 'pickup_lng', 'dropoff_lat', 'dropoff_lng', 'user_id',
       'driver_id', 'ride_distance', 'canceled_by_client',
       'canceled_by_driver', 'ride_to_suburb', 'weekend'],
      dtype='object')

In [ ]:
feature_frame = features[['ride_id', 
       'weekend']]

In [122]:
feature_frame = features.reset_index()
feature_frame = feature_frame[['week_day', 'month', 'year', 'hexagon_id', 'ride_id']]
feature_frame.columns = ['week_day', 'month', 'year', 'hexagon_id', 'num_in']
feature_frame['weekend'] = [True if weekday in (5, 6) else False for weekday in feature_frame.week_day.values]
feature_frame = features.ride_distance
# TODO: repalce small_frame on features
feature_frame = small_frame.groupby(['week_day', 'month', 'year', 'hexagon_id']).sum()['ride_distance']

In [119]:
feature_frame.shape

(3394, 6)

In [123]:
feature_frame.head()

week_day  month  year    hexagon_id
0.0       1.0    2019.0  1.0           244.559999
                         2.0             3.110000
                         3.0            30.570001
                         4.0            45.670000
                         5.0            40.340000
Name: ride_distance, dtype: float64